Before we begin, let us execute the below cell to display information about the NVIDIA® CUDA® driver and the GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by clicking on it with your mouse, and pressing Ctrl+Enter, or pressing the play button in the toolbar above. You should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Exercise 5

###  Learning objectives

The **goal** of this lab is to:

- Learn how to inspect the application's kernels with NVIDIA Nsight™ Compute
- Learn how to execute rules inside the Nsight Computer profiler and find bottlenecks
- Learn how to add baselines and compare results/reports

We do not intend to cover:

- The OpenACC programming model
- Advanced optimization techniques in detail

As mentioned earlier on, Nsight Compute and Nsight Systems each serve a different purpose in profiling and with different functionalities. In previous exercises, we inspected the timelines, measured activity durations, and tracked CPU events via the Nsight Systems profiler. The purpose of this exercise is to get familiar with the Nsight Compute tool. This tool provides access to kernel-level analysis using GPU performance metrics.

We first profile the GPU application and identify certain areas in the code, that don't behave as expected. Then we isolate those kernels and profile them via Nsight Compute. 

**Understand and analyze** the code present at:

[OpenACC Code](../source_code/lab5/miniWeather_openacc.cpp) 

Open the downloaded file for inspection. Once done, **Compile** the code with `make` and **Profile** it with `nsys`.

In [ ]:
!cd ../source_code/lab5 && make clean && make

Now, **Profile** the code with Nsight System command line interface (CLI):

In [ ]:
!cd ../source_code/lab5 && nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o miniWeather_5 ./miniWeather

Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> [Here](../source_code/lab5/miniWeather_5.nsys-rep), then choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System user interface(UI) locally. As shown in the example output, the initialization looks very expensive and the kernels are very small meaning that the GPU compute part of the problem is very small. Check how much time (what percentage) is spent in each kernel relative to the time it takes to run the code. 

<img src="images/5_init.png" width="80%">

From the "Timeline" view, inspect the less efficient kernel. Next, inspect the most expensive kernel and see what the Nsight Compute recommends.

<img src="images/cexer5.png" width="80%">

Now, **Profile** the application via Nsight Compute CLI (`ncu`): 

In [ ]:
!cd ../source_code/lab5 && ncu --set full -k regex:compute_tendencies_x --launch-skip 10 --launch-count 1 -f -o miniWeather1 ./miniWeather

Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> [Here](../source_code/lab5/miniWeather1.ncu-rep), then choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI. This tool has many sections that focus on different areas of the GPU and presents them all on one page. 

**Note:** If you do not specify a specific kernel name when profiling, all kernels will be profiled and will slow down the profiling time.

<img src="images/ccompute.png">

The "GPU Speed Of Light Throughput" section shows less than 1% Compute (SM) Throughput. As you can see from the example output below, the Nsight Compute profiler suggests looking at the "Launch Statistics" section because the kernel grid is too small to fill the available resources on the GPU. 

We previously discussed Amdahl's law in the first exercise. It is very important to understand the relation between the problem size and computational performance as this can determine the amount of speedup and benefit you get by parallelizing on GPU. Due to the small problem size (`nx_glob`, `nz_glob` , and `sim_time` in this example), most of the computation is dominated by the initialization and there is not enough work/computation to make it suitable for GPU. Run the application with different values `nx_glob`, `nz_glob` , and `sim_time` and profile the same kernel (`nx_glob` = 400 , `nz_glob`= 200 , and `sim_time`= 100). 

In [ ]:
!cd ../source_code/lab5 && make clean && make

In [ ]:
!cd ../source_code/lab5 && ncu --set full -k regex:compute_tendencies_x --launch-skip 100 --launch-count 1 -f -o miniWeather2 ./miniWeather 400 200 100

Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> [Here](../source_code/lab5/miniWeather2.ncu-rep), then choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI. 

**Diff the reports**

Open both reports via the Nsight Compute UI. From the top of the first report, click on *Add Baseline*, then do the same for the second report shown in the second tab. Have a look at the expected output:

<img src="images/c2compute.png">

You can now compare the two reports and see how changes you made to the cell size, affected specific metrics by looking at each section. By increasing the cell size, we increased the "Compute Throughput" and "Memory Throughput" by 35% and 26% respectively. 


Next, look at the Roofline chart which shows a high-level overview of the utilization for compute and memory resources of the GPU. We can see that increasing cell sizes resulted in the performance getting  closer to the Rooflines. 

<img src="images/roofline.png">

However, the kernel is still too small to utilize the GPU and "Compute" and "Memory" are still less than 50% utilized. The "GPU Speed of Light Throughput" section, gives a high-level overview of the throughput for compute and memory resources of the GPU for each unit. Based on this information we can find the performance limiters and categorize them into four possible combinations:

- Compute Bound: SM>50% & Mem<50%
- Bandwidth Bound: SM<50% & Mem>50%
- Latency Bound: SM<50% & Mem<50%
- Compute and Bandwidth Bound : SM>50% & Mem>50%


According to the Roofline, this kernel is fp64 bound, and we should consider using 32-bit precision floating point operations to improve its performance. 

The detailed "Memory Workload Analysis" section shows all the data traffic between various stages of the GPU and what your kernel is actually transferring. This section suggests that we need to look at the memory access pattern in the code as the load/store pattern is not optimal. The solution is to minimize how many cache lines need to be accessed per memory  request. 


This is out of scope for this tutorial but  you can have a look at the algorithm and see if you can change anything to do more work per memory access.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommended you go to your browser's file menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip file of the files you have been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f _profiler_files.zip
zip -r _profiler_files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> [Here](../_profiler_files.zip), then choosing <mark>Save Link As</mark>. 

-----

# <div style="text-align: center ;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em">[HOME](../_start_profiling.ipynb)</div>

-----

# Links and Resources

[OpenACC API Guide](https://www.openacc.org/sites/default/files/inline-files/OpenACC%20API%202.6%20Reference%20Guide.pdf)

[NVIDIA Nsight Compute](https://docs.nvidia.com/nsight-compute/index.html)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System and Compute profiler outputs, please download the latest versions from below pages:

- https://developer.nvidia.com/nsight-systems
- https://developer.nvidia.com/nsight-compute


Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.